<a href="https://colab.research.google.com/github/pj0620/google-colab-notebooks/blob/main/Minesweeper_rl.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import numpy as np
import random
from scipy.signal import convolve2d
%pip install stable-baselines3[extra]

import numpy as np
import gymnasium as gym
from gymnasium import spaces

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 434.7/434.7 kB 11.0 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Using cached Farama_Notifications-0.0.4-py3-none-any.whl.metadata (558 bytes)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 953.9/953.9 kB 31.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 182.3/182.3 kB 11.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 44.0 MB/s eta 0:00:00
Using cached Farama_Notifications-0.0.4-py3-none-any.whl (2.5 kB)
  Created wheel for AutoROM.accept-rom-license: filename=AutoROM.accept_rom_license-0.6.1-py3-none-any.whl size=446660 sha256=8efaf1b46efc58156f00acca9479a2ef7f7acea962b4e24062e66109026ca312
  Stored in directory: /root/.cache/pip/wheels/6b/1b/ef/a43ff1a2f1736d5711faa1ba4c1f61be1131b8899e6a057811
Successfully built AutoROM.accept-rom-license


In [1]:
TOTAL_BOMBS = 10
BOARD_SIZE = 10

KERNAL = np.ones((3, 3))

def get_bombs(board_size, total_bombs):
  bombs = np.zeros(shape=(board_size, board_size))
  placed_bombs = 0
  while placed_bombs < total_bombs:
    i = random.randint(0, board_size-1)
    j = random.randint(0, board_size-1)

    if bombs[i][j] == 0:
      bombs[i][j] = 1
      placed_bombs += 1
  return bombs

def get_values(bombs):
  KERNAL = np.ones((3, 3))
  return convolve2d(bombs, KERNAL, mode='same')

def get_representation(vals, visible):
  return vals * visible, visible


NameError: name 'np' is not defined

## Custom gymnasium env

In [52]:
import gymnasium as gym
import numpy as np
from gymnasium import spaces

class MinesweeperEnvironment(gym.Env):
  # Because of google colab, we cannot implement the GUI ('human' render mode)
  metadata = {"render_modes": ["console"]}

  def __init__(self, board_size=10, total_bombs=10, render_mode="console"):
    super().__init__()
    self.render_mode = render_mode
    self.board_size = board_size
    self.total_bombs = total_bombs

    self.visible = np.zeros((self.board_size, self.board_size), dtype=np.int8)
    self.set_bombs()
    self.set_values()

    # Define observation space
    self.observation_space = spaces.Dict({
      "visible_vals": spaces.Box(low=0, high=8, shape=(self.board_size, self.board_size), dtype=np.int8),
      "visible": spaces.Box(low=0, high=1, shape=(self.board_size, self.board_size), dtype=np.int8)
    })

    self.observation_space = spaces.Box(
        low=0, high=8, shape=(2 * self.board_size**2,), dtype=np.int8
    )

    # Define action space
    self.action_space = spaces.Discrete(self.board_size**2)

  def set_bombs(self):
    self.bombs = np.zeros(shape=(self.board_size, self.board_size),  dtype=np.int8)
    placed_bombs = 0
    while placed_bombs < self.total_bombs:
      i = random.randint(0, self.board_size-1)
      j = random.randint(0, self.board_size-1)

      if self.bombs[i][j] == 0:
        self.bombs[i][j] = 1
        placed_bombs += 1

  def set_values(self):
    KERNAL = np.ones((3, 3))
    self.vals = convolve2d(self.bombs, KERNAL, mode='same').astype(np.int8)

  def reset(self, seed=None, options=None):
    # Reset the environment to an initial state
    self.visible = np.zeros((self.board_size, self.board_size), dtype=np.int8)
    self.set_bombs()
    self.set_values()

    return self.get_state(), {}

  def step(self, action):
    # Implement the logic for taking a step in the environment
    x = action // self.board_size
    y = action % self.board_size

    if self.bombs[x][y] == 1:
      return self.get_state(), -10, True, True, {}

    self.visible[x][y] = 1
    visible_cells = np.sum(self.visible)
    done = (self.board_size**2 - visible_cells) == self.total_bombs
    return self.get_state(), float(visible_cells), done, False, {}

  def get_state(self):
    return np.hstack((np.ravel(self.visible * self.vals), np.ravel(self.visible)))

  def render(self, mode='human'):
    # Optionally implement visualization
    pass

from stable_baselines3.common.env_checker import check_env

env = MinesweeperEnvironment(board_size=10, total_bombs=10)

# Do
check_env(env)
obs, info = env.reset()
n_steps = 10
for _ in range(n_steps):
    # Random action
    action = env.action_space.sample()
    obs, reward, terminated, truncated, info = env.step(action)
    if terminated:
        obs, info = env.reset()
env.reset()

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


AssertionError: The `terminated` signal must be a boolean